In [626]:
import pandas as pd
import numpy as np
import math as mt

pi=np.pi

In [627]:
#def blocks
def meter(x):
    return x * 0.0254

def approx(num):
    remainder = num % 20
    lower_multiple = num - remainder
    higher_multiple = lower_multiple + 20
    return int(lower_multiple), int(higher_multiple)

def prop(df1, index):
    Cp = df1.Specific_heat[index] #Specific heat value
    k = df1.thermal_conductivity[index] #Specific heat value
    Pr = df1.Prandtl_Number[index] #Prandtl_Number
    ρ = df1.density[index]
    ν = df1.kinematic_viscosity[index] * (10 ** -6)
    return Cp, k, Pr, ρ, ν

def avg(y1, y2, x, x1, x2):
    return y1 + ((y2 - y1)/(x2 - x1)*(x-x1))

def prop2(df1, temp):
    lower, upper = approx(temp)

    for i in range(len(df1.density)):
        if lower == df1.temperature[i]:
            index1 = i
            index2 = i + 1

    a1, a2, a3, a4, a5 = prop(df1, index1)
    b1, b2, b3, b4, b5 = prop(df1, index2)

    c1 = avg(a1, b1, temp, lower, upper)
    c2 = avg(a2, b2, temp, lower, upper)
    c3 = avg(a3, b3, temp, lower, upper)
    c4 = avg(a4, b4, temp, lower, upper)
    c5 = avg(a5, b5, temp, lower, upper)
    
    return c1, c2, c3, c4,c5

def D_hydraulic(Di_m, do_m):
    return float(Di_m - do_m)
def A_cold(di_m):
    return float(pi * (di_m ** 2 ) / 4)
def A_hot(do_m, Di_m):
    return float(pi*((Di_m**2) - (do_m**2))/ 4)
def V_hot(mh, ρ_h, A_h) :
    return float(mh/(ρ_h*A_h))
def V_cold (mc, ρ_c1, A_c) :
    return float(mc/(ρ_c1*A_c))
def Re_hot(V_h, D_Hyd, ν_h) :
    return float((V_h * D_Hyd) / ν_h)
def Re_cold(V_c, di_m, ν_c1) :
    return float((V_c*di_m)/ν_c1)
def Nusselt_h(Re_h, Pr_h) :
    return float(0.023 * (Re_h ** 0.8) * (Pr_h ** 0.4))
def Nusselt_c(Re_c, Pr_c1) :
    return float(0.023 * (Re_c ** 0.8) * (Pr_c1 ** 0.3))
def D_equivalent (Di_m, do_m) :
    return float(((Di_m**2) - (do_m**2)) / do_m)
def h_hot (Nusselt_h, k_h, D_e) :
    return float((Nusselt_h * k_h) / D_e)
def h_cold (Nusselt_c, k_c1, di_m) :
    return float((Nusselt_c * k_c1) / di_m)
def R_equivalent (h_h, h_c, do_m, di_m) :
    return float((1/(h_h*pi*do_m)) + (1/(h_c*pi*di_m)) + (1/(2*pi*k_mat))*(mt.log(do_m/di_m)))
def U_coeff (R_e, do_m) :
    return float(1/(R_e*pi*do_m))
def C_p (mh, Cp_h, mc, Cp_c1) :
    return float(min(mh*Cp_h, mc*Cp_c1))
def C_p1 (mh, Cp_h, mc, Cp_c1) :
    return float(max(mh*Cp_h, mc*Cp_c1))
def HT (E, C, thi, tci) :
    return float(E*C*(thi-tci))
def th_o (thi, Q, mh, Cp_h) :
    return float(thi - (Q/(mh*Cp_h)))
def tc_o (tci, Q, mc, Cp_c1) :
    return float(tci + (Q/(mc*Cp_c1)))
def T_LMTD (tho, thi, tco, tci) :
    return float(((tho-tci)-(thi-tco))/mt.log((tho-tci)/(thi-tco)))
def area (Q,t_lmtd, U) :
    return float(Q/(U*t_lmtd))
def NTU (U, A, C_min) :
    return float(U*A / C_min)
def C_ratio (C_min, C_max) :
    return float(C_min/C_max)

In [628]:
#Given Values
thi = 100 #Temperature hot inlet
tci = 30 #Temperaturecold inlet
E = 0.75 #Effectiveness Assumed
thi_prev = 0
tci_prev = 0
i = 1
j = 0

mh = 10 #Mass Flow Rate of hot water
mc = 10 #Mass flow rate of cold water
Di_inch = 2 #Diameter of outer pipe in inch
Di_m = meter (Di_inch) #Diameter of outer pipe in meter
di_inch = 0.68 #Inner diameter of inner pipe
di_m = meter (di_inch) #Inner diameter of the inner pipe in meter
do_inch = 0.75 #Outer diameter of inner pipe
do_m = meter (do_inch) #Outer diameter of the inner pipe in meter
k_mat = 13.6 #Material of HX is SS316
R_fo = 0.001754
R_fi = 0.0000827

In [629]:
#Constants
D_Hyd = D_hydraulic(Di_m, do_m)
A_c = A_cold(di_m)
A_h = A_hot(do_m, Di_m)
D_e = D_equivalent (Di_m, do_m)

In [630]:
data = {
    "temperature" : [0,20,40,60,80,100,120,140,160,180,200,220,240,260,280,300],
    "density" : [1002, 1000, 995, 985, 974, 961, 945, 928, 909, 889, 867, 842, 815, 786, 752, 714],
    "kinematic_viscosity" : [1.788, 1.006,0.657, 0.478, 0.364, 0.293, 0.247, 0.213, 0.189, 0.173, 0.160, 0.149, 0.143,0.137, 0.135,0.135],
    "Specific_heat" : [4216, 4178, 4178, 4183, 4195, 4216, 4250, 4283, 4342, 4417, 4505, 4610, 4756, 4949, 5208, 5728],
    "thermal_conductivity" : [0.5524, 0.5978, 0.6280, 0.6513, 0.6687, 0.6804, 0.6850, 0.6838, 0.6804, 0.6757, 0.6652, 0.6524, 0.6350, 0.6106, 0.5803, 0.5396],
    "Prandtl_Number" : [13.6, 7.02, 4.34, 3.02, 2.22, 1.74, 1.446, 1.241, 1.099, 1.044, 0.937, 0.891, 0.871, 0.874, 0.910, 1.019]
}
df = pd.DataFrame(data)
df

,temperature,density,kinematic_viscosity,Specific_heat,thermal_conductivity,Prandtl_Number
0,0,1002,1.788,4216,0.5524,13.600
1,20,1000,1.006,4178,0.5978,7.020
2,40,995,0.657,4178,0.6280,4.340
3,60,985,0.478,4183,0.6513,3.020
4,80,974,0.364,4195,0.6687,2.220
5,100,961,0.293,4216,0.6804,1.740
6,120,945,0.247,4250,0.6850,1.446
7,140,928,0.213,4283,0.6838,1.241
8,160,909,0.189,4342,0.6804,1.099
9,180,889,0.173,4417,0.6757,1.044


In [631]:
while (i > 0.001): 
    index = 22
    for i in range(len(df.temperature)):
        if (thi == df.temperature[i]):
            index = i

    if (index != 22):
        Cp_h, k_h, Pr_h, ρ_h, ν_h = prop(df,index)

    else :
        Cp_h, k_h, Pr_h, ρ_h, ν_h = prop2(df,thi)

    index_1 = 22
    for i in range(len(df.temperature)):
        if (tci == df.temperature[i]):
            index_1 = i   

    if (index_1 != 22):
        Cp_c1, k_c1, Pr_c1, ρ_c1, ν_c1 = prop(df,tci)

    else:
        Cp_c1, k_c1, Pr_c1, ρ_c1, ν_c1 = prop2(df, tci)

    V_h = V_hot(mh, ρ_h, A_h)
    V_c = V_cold (mc, ρ_c1, A_c)

    Re_h = Re_hot(V_h, D_Hyd, ν_h)
    Re_c = Re_cold(V_c, di_m, ν_c1)

    Nusselt_h = 0.023 * (Re_h ** 0.8) * (Pr_h ** 0.4)
    Nusselt_c = 0.023 * (Re_c ** 0.8) * (Pr_c1 ** 0.3)

    h_h = h_hot (Nusselt_h, k_h, D_e)
    h_c = h_cold (Nusselt_c, k_c1, di_m)

    R_e = R_equivalent (h_h, h_c, do_m, di_m)
    U = U_coeff (R_e, do_m) 

    C_min = C_p (mh, Cp_h, mc, Cp_c1)
    C_max = C_p1 (mh, Cp_h, mc, Cp_c1)
    Q = HT (E, C_min, thi, tci)
    tho = th_o (thi, Q, mh, Cp_h)
    tco = tc_o (tci, Q, mc, Cp_c1)
    t_lmtd = T_LMTD (tho, thi, tco, tci)

    A = area (Q,t_lmtd, U)
    N = NTU (U, A, C_min)
    C= C_ratio (C_min, C_max)
    E1 = (1-np.exp(-N * (1 - C)))/(1 - (C * mt.exp(-N*(1-C))))
    l_total = round(A / (pi * di_m ), 2)
    passes = round(l_total / 4, 0)
   
    thi_prev = thi
    tci_prev = tci
    thi = (thi + tho) / 2
    tci = (tci + tco) / 2
    E = E1

    i = thi_prev - thi
    j = j + 1


In [632]:
print(thi)
print(tho)
print(tci)
print(tco)
print(E)
print(h_h)
print(h_c)
print(l_total)
print(passes)

0.00040783335110461394
65.12640493128387
65.12599709793277
65.12613304227719
65.12654087569187
9
0.7500000000890277
6101.858118940093
113407.90184687513
560.62
140.0


In [633]:
A = (1 / h_h) + R_fo + (do_m / 2*k_mat) * mt.log(do_m / di_m) + (do_m * R_fi) / di_m + do_m / (di_m * h_c)
U_out = 1 / A

In [634]:
B = (1 / h_c) + R_fi + (di_m / 2*k_mat) * mt.log(do_m / di_m) + (di_m * R_fo) / do_m + di_m / (do_m * h_h)
U_in = 1 / B

In [635]:
A_out = Q / (U_out * t_lmtd)
A_in = Q / (U_in * t_lmtd)
C = A_out / A_in
SOD = (C - 1) * 100
Q_round = round(Q, 2)
U_round = round(U, 2)
t_lmtd_round = round(t_lmtd, 2)
SOD_round = round(SOD, 2)
E_round = round(E, 2)

In [636]:
print(thi)
print(tho)
print(tci)
print(tco)
print(j)
print(E)
print(h_h)
print(h_c)

65.12640493128387
65.12599709793277
65.12613304227719
65.12654087569187
9
0.7500000000890277
6101.858118940093
113407.90184687513


In [637]:
print ("The performance parameters of this heat exchanger are:") 
print (f"The Heat Transfer Rate is {Q_round} W,")
print (f"The Effectiveness is {E_round},")
print (f"Overall Heat Transfer Coefficient is {U_round} W/m2K,")
print (f"The total length is {l_total}m and it has {passes} total passes of length 4m per pass,")
print (f"and the Surface over design is {SOD_round} %.")

The performance parameters of this heat exchanger are:
The Heat Transfer Rate is 34.14 W,
The Effectiveness is 0.75,
Overall Heat Transfer Coefficient is 4128.27 W/m2K,
The total length is 560.62m and it has 140.0 total passes of length 4m per pass,
and the Surface over design is 10.29 %.


In [638]:
  if passes >= 50:
    print(passes, "passes are practically not possible. So rather than using a double pipe heat exchanger, we suggest using either a Shell and Tube heat exchanger or Plate heat exchanger.")
    print('\nBased on these parameters, a Plate Heat Exchanger (PHE) is the recommended choice due to the following technical advantages: \nFluid Compatibility: PHEs excel at transferring heat between low viscosity fluids like water due to their high heat transfer area per unit volume achieved by the thin, corrugated plates. \nPerformance: The desired effectiveness (75%) and moderate heat transfer rate (35W) are well-suited for a PHE design, which can achieve high effectiveness in a compact size. \nCounterflow Efficiency: PHEs inherently benefit from counterflow configurations. The large heat transfer area created by the plates maximizes heat exchange between the hot and cold fluids.')
    print('Pressure Drop: PHEs typically exhibit lower pressure drops compared to Shell and Tube Heat Exchangers (SHTEs) with similar effectiveness. This is crucial if minimizing pressure loss is important in your system. \nMaintainability: The accessible plate design of a PHE facilitates easier cleaning, which can be advantageous if there\'s a risk of water scaling within the exchanger. \nCost and Size: For smaller heat transfer duties like 35W PHEs are generally more compact and cost-effective solutions compared to SHTEs.')
    print('Potential Considerations for Shell and Tube Heat Exchangers (SHTEs). \nWhile a PHE is the preferred choice in most cases for your scenario, there are some niche applications where a SHTE might offer specific advantages:')
    print('High-Pressure Applications: If your system operates under extremely high pressures that a PHE might not be able to handle, a SHTE could be a consideration due to their superior pressure tolerance. \nFouling Resistance: In cases with a high risk of fouling, particularly in the hot water stream, a SHTE might be a better option in some scenarios. The tube design in a SHTE allows for easier mechanical cleaning methods compared to a PHE. However, a thorough evaluation of the specific fouling characteristics and cleaning procedures is necessary.')
    print('\nImportant Considerations for SHTEs:')
    print('Pressure Drop: Remember that SHTEs typically have a higher pressure drop compared to PHEs. Ensure your system can accommodate the additional pressure loss. \nMaintenance: Cleaning a SHTE can be more complex due to the enclosed tube bundle. Evaluate the maintenance requirements and downtime associated with cleaning an SHTE compared to a PHE. \nCost and Size: For a smaller heat transfer duty like 35W, SHTEs might be less cost-effective and more bulky compared to PHEs.')
    print('\nConclusion')
    print('For the scenario you described water, counterflow, 35W, a PHE remains the most suitable choice due to its advantages in pressure drop, maintainability, size, and potentially cost. However, if you have specific requirements for extremely high pressures or a severe fouling concern that cannot be addressed by a PHE, consulting a heat exchanger specialist is recommended to explore the feasibility of a SHTE and weigh the trade-offs involved.')

140.0 passes are practically not possible. So rather than using a double pipe heat exchanger, we suggest using either a Shell and Tube heat exchanger or Plate heat exchanger.

Based on these parameters, a Plate Heat Exchanger (PHE) is the recommended choice due to the following technical advantages: 
Fluid Compatibility: PHEs excel at transferring heat between low viscosity fluids like water due to their high heat transfer area per unit volume achieved by the thin, corrugated plates. 
Performance: The desired effectiveness (75%) and moderate heat transfer rate (35W) are well-suited for a PHE design, which can achieve high effectiveness in a compact size. 
Counterflow Efficiency: PHEs inherently benefit from counterflow configurations. The large heat transfer area created by the plates maximizes heat exchange between the hot and cold fluids.
Pressure Drop: PHEs typically exhibit lower pressure drops compared to Shell and Tube Heat Exchangers (SHTEs) with similar effectiveness. This is c